# Project Pipeline
Execute the cells step by step to obtain a prediction and score for your configuration.
In order to make the widgets work you might need to execute 
```
jupyter nbextension enable --py --sys-prefix widgetsnbextension
```
on your system.

## 1. Imports

In [1]:
import importlib
import data_utils
import widget_ui as ui

# Load potential changes in the modules during development
data_utils = importlib.reload(data_utils)
ui = importlib.reload(ui)

## 2. Prepare Data Sets
### 2.1 Load Data
Load training and test data. Furthermore, create a sample set for quickly calculating transformations such as PCA.

In [2]:
# These original data sets should remain unchanged and serve as a backup
# for resetting modified (e.g. subsampled or projected) data sets.
orig_data = data_utils.load_from('data/all_train.csv', 'data/all_test.csv')
orig_sample_data = data_utils.load_sample_data('data/all_sample.csv')

# Data sets that can be modified
data = orig_data.copy()
sample_data = orig_sample_data.copy()

### 2.2 Subsample Data
Sample the specified rate from the whole data set for test and training individually to reduce the data set size for the following steps.

In [3]:
ui.subsampling_ui(orig_data=orig_data, output_data=data)

## 3. Preprocessing

In [4]:
preprocessors = {}

### 3.1 Remove Correlations
Find correlated pairs of features and remove one feature of these two, respectively.

In [5]:
ui.correlations_ui(preprocessors, sample_data)

### 3.2 PCA
Compute PCA on the sample data set.

In [6]:
ui.pca_ui(preprocessors, sample_data)

## 4. Configure Classifiers

In [7]:
classifiers = {}

### 4.1 Multilayer Perceptron

In [8]:
ui.mlp_ui(classifiers)

### 4.2 Naive Bayes

In [9]:
ui.nb_ui(classifiers)

## 5. Training

In [10]:
ui.training_ui(data, sample_data, preprocessors, classifiers)

## 6. Prediction

In [11]:
ui.prediction_ui(data, sample_data, preprocessors, classifiers)